# Notæ 1: Problema de las ocho reinas

### En esta nota, veremos la creación y optimización de un modelo sencillo. Recuerde que cada vez que quiera optimizar el modelo, deberá reiniciar el Kernel de Python (ir al menú "Kernel" > "Restart").

In [ ]:
# Comenzamos solicitando las librerías necesarias.
import cplex, chess
from docplex.mp.model import Model

In [ ]:
# Ahora creamos el modelo.
p = cplex.Cplex()
m = Model(name='queens')

#### Las variables de decisión son:
- x[i,j] = 1 si se coloca una reina en la posicion (i,j), 0 si no, para todo i, j en T

#### La función objetivo es:
- max sum_ij x[i,j]

#### Y las restricciones son:
- Para dada una posición (i,j), colocar una reina en esa posición impide la colocación de una reina en cualquier otra posición tanto en sentido horizontal (es decir, otra (k, j) con k en T-{i}) como vertical (es decir, otra (i, k) con k en T-{j}) como diagonal, la que se puede modelar como una big-M, por ejemplo para (1,1) tenemos:
50 x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] + x[1,6] + x[1,7] + x[1,8] + x[2,1] + x[2,2] + x[3,1] + x[3,3] + x[4,1] + x[4,4] + x[5,1] + x[5,5] + x[6,1] + x[6,6] + x[7,1] + x[7,7] + x[8,1] + x[8,8] <= 50

In [ ]:
# Conjuntos útils
T = range(1, 9)    # T = {1, 2, ..., 8}
S = {k for k in range(-7, 8) if k != 0} # S = {-7, -6, ..., -1, 1, 2, ..., 7}

# Creamos las variables
x = {(i,j): m.binary_var(name='x_{0}_{1}'.format(i,j)) for i in T for j in T}

# Creamos la función objetivo
m.maximize(m.sum(x[i,j] for i in T for j in T))

# Listamos las restricciones
for i in T:
    for j in T:
        m.add_constraint(50 * x[i,j]
                         + m.sum(x[k,j] for k in T if k != i)
                         + m.sum(x[i,k] for k in T if k != j)
                         + m.sum(x[i+k,j+k] for k in S if i+k in T and j+k in T)
                         + m.sum(x[i+k,j-k] for k in S if i+k in T and j-k in T) <= 50)

# Observamos algunos datos (deberían ser 64 variables y 64 restricciones)
m.print_information()

In [ ]:
# Para revisar que el modelo esté ok, podemos exportarlo a un archivo y revisarlo con un editor.
m.export_as_lp("modelito.lp")

In [ ]:
# Resolvemos y mostramos la solución numérica

sol = m.solve(log_output=True)
print('')
if sol is None:
    print('El modelo es infactible!')
else:
    print('Solución:')
    sol.display()

In [ ]:
# Veamos la solución en un Tablero
board = chess.Board()
board.clear()
for i in T:
    for j in T:
        if sol.get_value(x[i,j]) > 0.5:
            squareIndex=i*8+j-9
            board.set_piece_at(chess.SQUARES[squareIndex], chess.Piece(chess.QUEEN, chess.WHITE))
board

#### Actividades:
- Añada una restricción que obligue a colocar una posición en algún lugar fijo del tablero. ¿Sigue siendo 8 la solución óptima? ¿Puede forzar a que sea inferior a ese número?
- Añada una o varias restricciones para tornar al modelo infactible y compruébelo.
- ¿Se anima a re-escribir el modelo como un Problema del Conjunto Estable, es decir reemplazando las actuales restricciones por otras de la forma x[i,j] + x[k,l] <= 1 (formulación EDGE)?